In [4]:
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import pickle
from ast import literal_eval
from tqdm import tqdm
import numpy as np

In [5]:
def recommend_items(input_items, rules, top_n=5, metric='confidence'):
    recommendations = {}
    for i , (antecedent, consequent, antecedent_support, consequent_support, support, confidence, lift, leverage, conviction, zhangs_metric) in rules.iterrows():
        if antecedent.issubset(input_items) and not consequent.issubset(input_items):
            for item in consequent:
                if item not in input_items:
                    if item not in recommendations:
                        recommendations[item] = []
                    recommendations[item].append((confidence, support, lift, leverage, conviction, zhangs_metric))
    # Average the confidence and support values for the same item
    recommendations = {
        item: (
            sum(conf for conf, _, _, _, _, _ in item_rules) / len(item_rules), 
            sum(sup for _, sup, _, _, _, _ in item_rules) / len(item_rules), 
            sum(lift for _, _, lift, _, _, _ in item_rules) / len(item_rules), 
            sum(leverage for _, _, _, leverage, _, _ in item_rules) / len(item_rules),
            sum(conviction for _, _, _, _, conviction, _ in item_rules) / len(item_rules), 
            sum(zhangs_metric for _, _, _, _, _, zhangs_metric in item_rules) / len(item_rules)
                )
        for item, item_rules in recommendations.items()
    }
    
    if metric == 'confidence':
        # Sort the recommendations by decreasing confidence and support
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][0], -x[1][1]))
    elif metric == 'support':
        # Sort the recommendations by decreasing support and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][1], -x[1][0]))
    elif metric == 'lift':
        # Sort the recommendations by decreasing lift and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][2], -x[1][0]))
    elif metric == 'leverage':
        # Sort the recommendations by decreasing leverage and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][3], -x[1][0]))
    elif metric == 'conviction':
        # Sort the recommendations by decreasing conviction and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][4], -x[1][0]))
    elif metric == 'zhangs_metric':
        # Sort the recommendations by decreasing Zhang's metric and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][5], -x[1][0]))
    else:
        raise ValueError('Invalid metric: {}'.format(metric))

    return [item for item, _ in sorted_recommendations[:top_n]]

def evaluate_recommendations(test_data, user_items, rules, top_n, metric='confidence'):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for user, true_items in tqdm(test_data.items()):
        # Assuming user_items is a dictionary with user IDs as keys and their associated items as values
        input_items = user_items[user]
        # Get recommendations for the user
        recommended_items = set(recommend_items(input_items, rules, top_n=top_n, metric=metric))
        true_items = set(true_items)
        true_positives += len(recommended_items.intersection(true_items))
        false_positives += len(recommended_items - true_items)
        false_negatives += len(true_items - recommended_items)
    # Calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1_score

def create_test_data(baskets, test_size=0.1):
    '''
    Create test data from the given baskets - drop items from the baskets and add them to the test data - see how well the model can predict them

        :param user_items: dictionary with user IDs as keys and their associated items as values
        :param test_size: size of the test data

    :return: 
        train_data - dictionary with transaction IDs as keys and their associated items as values 
        test_data - dictionary with transaction IDs as keys and their associated items as values - for testing the model (items that were left in the basket)
        test_labels - dictionary with transaction IDs as keys and their associated items as values - for testing the model (items that were dropped from the basket)
    '''

    train_data = {basket_id : items for basket_id, items in enumerate(baskets) if np.random.rand() > test_size}
    test_labels = {}
    test_data = {}

    for id, basket in enumerate(baskets):
        if id not in train_data:
            n = len(basket)
            basket = list(basket)
            if n > 1:
                # n_to_drop = np.random.randint(1, n // 2 + 1)  # drop at least one item, but not more than half of the items
                n_to_drop = 1
                items_to_drop = np.random.choice(basket, n_to_drop, replace=False)  # choose items to drop
                basket = set(basket) - set(items_to_drop)  # remove items from the basket
                test_labels[id] = set(items_to_drop)  # add items to the test data
                test_data[id] = basket  # update user items
    
    return train_data, test_data, test_labels

def play_with(frequent_itemsets, metrics, test_data, test_labels, top_n=5):
    '''
    Play with different metrics and thresholds for the creation of rules

    :param frequent_itemsets: frequent itemsets 
    :param methods: dictionary of methods to use for the creation of rules mapped to their thresholds

    :return: dataframe of methods, thresholds, precision, recall, f1_score
    '''
    results = []
    if type(top_n) == list:
        for n in top_n:
            for metric, threshold_list in metrics.items():
                loop = tqdm(threshold_list)
                for threshold in loop:
                    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=threshold)
                    loop.set_description(f'Method: {metric}, Threshold: {threshold:.4f}, N_rules: {len(rules)}')
                    precision, recall, f1_score = evaluate_recommendations(test_data, test_labels, rules, top_n=n, metric=metric)
                    results.append((metric, threshold, n, precision, recall, f1_score))
        return pd.DataFrame(results, columns=['method', 'threshold', 'top_n', 'precision', 'recall', 'f1_score'])
    else:
        for metric, threshold_list in metrics.items():
            loop = tqdm(threshold_list)
            for threshold in loop:
                rules = association_rules(frequent_itemsets, metric=metric, min_threshold=threshold)
                loop.set_description(f'Method: {metric}, Threshold: {threshold:.4f}, N_rules: {len(rules)}')
                precision, recall, f1_score = evaluate_recommendations(test_data, test_labels, rules, top_n=top_n, metric=metric)
                results.append((metric, threshold, precision, recall, f1_score))
        return pd.DataFrame(results, columns=['method', 'threshold', 'precision', 'recall', 'f1_score'])

In [6]:
# Use same as in task 2 for comparison purposes
with open('../assignment_02_task2/test_set.pkl', 'rb') as f:
    test = pickle.load(f)

with open('../assignment_02_task2/test_labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [11]:
# min_support ~ 0.0001
freq_itemsets = pickle.load(open('decoded_out_2_10.pkl', 'rb'))

methods_dict = {
    'support': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'confidence':[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'lift':[0, 0.5, 5, 10, 20, 50, 100],
    'leverage':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.001],
    'conviction':[0.5, 1, 1.1, 1.25, 1.4, 1.5, 2, 5],
    'zhangs_metric':[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1],
}

results = play_with(freq_itemsets, methods_dict, test, test_labels)

100%|██████████| 1461/1461 [02:15<00:00, 10.81it/s]9:  86%|████████▌ | 6/7 [51:20<06:03, 363.01s/it]
Method: confidence, Threshold: 1.0000, N_rules: 6279: 100%|██████████| 7/7 [53:35<00:00, 459.36s/it]
100%|██████████| 1461/1461 [08:50<00:00,  2.76it/s] 86%|████████▌ | 6/7 [1:27:04<13:36, 816.85s/it]
Method: lift, Threshold: 100.0000, N_rules: 24094: 100%|██████████| 7/7 [1:35:54<00:00, 822.02s/it]
100%|██████████| 1461/1461 [00:23<00:00, 61.36it/s]  83%|████████▎ | 5/6 [26:04<03:14, 194.39s/it]
Method: leverage, Threshold: 0.0010, N_rules: 1106: 100%|██████████| 6/6 [26:28<00:00, 264.69s/it]
100%|██████████| 1461/1461 [02:19<00:00, 10.49it/s]4:  88%|████████▊ | 7/8 [1:03:16<06:18, 378.27s/it]
Method: conviction, Threshold: 5.0000, N_rules: 6474: 100%|██████████| 8/8 [1:05:35<00:00, 491.96s/it]
100%|██████████| 1461/1461 [02:08<00:00, 11.40it/s]5962:  88%|████████▊ | 7/8 [1:47:19<14:56, 896.62s/it] 
Method: zhangs_metric, Threshold: 1.0000, N_rules: 5962: 100%|██████████| 8/8 [1:49:27<

In [12]:
display(results.sort_values(by='f1_score', ascending=False).head(10))
display(results.sort_values(by='recall', ascending=False).head(10))
display(results.sort_values(by='precision', ascending=False).head(10))

,method,threshold,precision,recall,f1_score
2,support,0.0010,0.050231,0.141956,0.074205
26,leverage,0.0010,0.049952,0.140153,0.073653
1,support,0.0005,0.043238,0.133844,0.065361
25,leverage,0.0005,0.043000,0.132041,0.064873
13,confidence,1.0000,0.047894,0.093285,0.063293
24,leverage,0.0004,0.039983,0.125732,0.060672
34,conviction,5.0000,0.043769,0.097341,0.060386
12,confidence,0.7500,0.043136,0.098693,0.060033
23,leverage,0.0003,0.035207,0.111762,0.053546
33,conviction,2.0000,0.036048,0.088779,0.051275


,method,threshold,precision,recall,f1_score
2,support,0.0010,0.050231,0.141956,0.074205
26,leverage,0.0010,0.049952,0.140153,0.073653
1,support,0.0005,0.043238,0.133844,0.065361
25,leverage,0.0005,0.043000,0.132041,0.064873
24,leverage,0.0004,0.039983,0.125732,0.060672
23,leverage,0.0003,0.035207,0.111762,0.053546
12,confidence,0.7500,0.043136,0.098693,0.060033
34,conviction,5.0000,0.043769,0.097341,0.060386
13,confidence,1.0000,0.047894,0.093285,0.063293
29,conviction,1.1000,0.031652,0.091032,0.046971


,method,threshold,precision,recall,f1_score
3,support,0.0050,0.051442,0.029743,0.037693
2,support,0.0010,0.050231,0.141956,0.074205
26,leverage,0.0010,0.049952,0.140153,0.073653
13,confidence,1.0000,0.047894,0.093285,0.063293
34,conviction,5.0000,0.043769,0.097341,0.060386
1,support,0.0005,0.043238,0.133844,0.065361
12,confidence,0.7500,0.043136,0.098693,0.060033
25,leverage,0.0005,0.043000,0.132041,0.064873
24,leverage,0.0004,0.039983,0.125732,0.060672
33,conviction,2.0000,0.036048,0.088779,0.051275


In [7]:
# min_support ~ 0.001
freq_itemsets = pickle.load(open('decoded_out_14_10.pkl', 'rb'))

methods_dict = {
    'support': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'confidence':[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'lift':[0, 0.5, 5, 10, 20, 50, 100],
    'leverage':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.001],
    'conviction':[0.5, 1, 1.1, 1.25, 1.4, 1.5, 2, 5],
    'zhangs_metric':[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1],
}

results2 = play_with(freq_itemsets, methods_dict, test, test_labels, top_n=[1, 2, 3, 4, 5])

100%|██████████| 1461/1461 [00:05<00:00, 243.97it/s]  86%|████████▌ | 6/7 [01:35<00:12, 12.02s/it]
Method: confidence, Threshold: 1.0000, N_rules: 270: 100%|██████████| 7/7 [01:41<00:00, 14.51s/it]
100%|██████████| 1461/1461 [00:22<00:00, 64.53it/s]  83%|████████▎ | 5/6 [02:01<00:24, 24.04s/it]
Method: leverage, Threshold: 0.0010, N_rules: 1048: 100%|██████████| 6/6 [02:23<00:00, 23.96s/it]
100%|██████████| 1461/1461 [00:07<00:00, 203.59it/s]  88%|████████▊ | 7/8 [01:54<00:12, 12.43s/it]
Method: conviction, Threshold: 5.0000, N_rules: 322: 100%|██████████| 8/8 [02:01<00:00, 15.16s/it]
100%|██████████| 1461/1461 [00:05<00:00, 249.82it/s]68:  88%|████████▊ | 7/8 [02:48<00:23, 23.01s/it]
Method: zhangs_metric, Threshold: 1.0000, N_rules: 268: 100%|██████████| 8/8 [02:54<00:00, 21.79s/it]
100%|██████████| 1461/1461 [00:05<00:00, 249.20it/s]  86%|████████▌ | 6/7 [01:36<00:12, 12.22s/it]
Method: confidence, Threshold: 1.0000, N_rules: 270: 100%|██████████| 7/7 [01:42<00:00, 14.68s/it]
100%|█

In [8]:
display(results2.sort_values(by='f1_score', ascending=False).head(10))
display(results2.sort_values(by='recall', ascending=False).head(10))
display(results2.sort_values(by='precision', ascending=False).head(10))

,method,threshold,top_n,precision,recall,f1_score
200,conviction,1.0000,5,0.060118,0.169896,0.088810
199,conviction,0.5000,5,0.060118,0.169896,0.088810
193,leverage,0.0001,5,0.058523,0.165390,0.086455
194,leverage,0.0002,5,0.058523,0.165390,0.086455
195,leverage,0.0003,5,0.058523,0.165390,0.086455
196,leverage,0.0004,5,0.058523,0.165390,0.086455
197,leverage,0.0005,5,0.058523,0.165390,0.086455
198,leverage,0.0010,5,0.058143,0.163137,0.085731
174,support,0.0010,5,0.057407,0.162235,0.084806
173,support,0.0005,5,0.057407,0.162235,0.084806


,method,threshold,top_n,precision,recall,f1_score
199,conviction,0.5000,5,0.060118,0.169896,0.088810
200,conviction,1.0000,5,0.060118,0.169896,0.088810
193,leverage,0.0001,5,0.058523,0.165390,0.086455
194,leverage,0.0002,5,0.058523,0.165390,0.086455
195,leverage,0.0003,5,0.058523,0.165390,0.086455
196,leverage,0.0004,5,0.058523,0.165390,0.086455
197,leverage,0.0005,5,0.058523,0.165390,0.086455
198,leverage,0.0010,5,0.058143,0.163137,0.085731
179,confidence,0.0100,5,0.057407,0.162235,0.084806
174,support,0.0010,5,0.057407,0.162235,0.084806


,method,threshold,top_n,precision,recall,f1_score
13,confidence,1.000,1,0.092562,0.025237,0.039660
34,conviction,5.000,1,0.089256,0.024335,0.038244
12,confidence,0.750,1,0.089256,0.024335,0.038244
4,support,0.010,1,0.085995,0.015773,0.026657
33,conviction,2.000,1,0.085950,0.023434,0.036827
11,confidence,0.500,1,0.085950,0.023434,0.036827
32,conviction,1.500,1,0.085526,0.023434,0.036788
3,support,0.005,1,0.082811,0.029743,0.043767
56,confidence,1.000,2,0.082645,0.045065,0.058326
55,confidence,0.750,2,0.081818,0.044615,0.057743


In [9]:
# min_support ~ 0.01
freq_itemsets = pickle.load(open('decoded_out_135_10.pkl', 'rb'))

methods_dict = {
    'support': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'confidence':[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'lift':[0, 0.5, 5, 10, 20, 50, 100],
    'leverage':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.001],
    'conviction':[0.5, 1, 1.1, 1.25, 1.4, 1.5, 2, 5],
    'zhangs_metric':[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1],
}

results3 = play_with(freq_itemsets, methods_dict, test, test_labels, top_n=[1, 2, 3, 4, 5])

100%|██████████| 1461/1461 [00:00<00:00, 250250.26it/s]|████▎     | 3/7 [00:00<00:01,  3.68it/s]
Method: confidence, Threshold: 1.0000, N_rules: 0: 100%|██████████| 7/7 [00:00<00:00,  7.75it/s]
100%|██████████| 1461/1461 [00:00<00:00, 233158.75it/s]|█▎        | 1/8 [00:00<00:02,  2.91it/s]
Method: conviction, Threshold: 5.0000, N_rules: 0: 100%|██████████| 8/8 [00:00<00:00, 20.15it/s]
100%|██████████| 1461/1461 [00:00<00:00, 268519.26it/s] 0%|          | 0/8 [00:00<?, ?it/s]
Method: zhangs_metric, Threshold: 1.0000, N_rules: 0: 100%|██████████| 8/8 [00:00<00:00, 117.20it/s]
100%|██████████| 1461/1461 [00:00<00:00, 265092.50it/s]|████▎     | 3/7 [00:00<00:01,  3.68it/s]
Method: confidence, Threshold: 1.0000, N_rules: 0: 100%|██████████| 7/7 [00:00<00:00,  7.70it/s]
100%|██████████| 1461/1461 [00:00<00:00, 230051.36it/s]|█▎        | 1/8 [00:00<00:02,  2.93it/s]
Method: conviction, Threshold: 5.0000, N_rules: 0: 100%|██████████| 8/8 [00:00<00:00, 19.79it/s]
100%|██████████| 1461/1461 [00:

In [10]:
display(results3.sort_values(by='f1_score', ascending=False).head(10))
display(results3.sort_values(by='recall', ascending=False).head(10))
display(results3.sort_values(by='precision', ascending=False).head(10))

,method,threshold,top_n,precision,recall,f1_score
173,support,0.0005,5,0.067157,0.032898,0.044162
187,lift,0.5000,5,0.067157,0.032898,0.044162
199,conviction,0.5000,5,0.067157,0.032898,0.044162
172,support,0.0001,5,0.067157,0.032898,0.044162
174,support,0.0010,5,0.067157,0.032898,0.044162
175,support,0.0050,5,0.067157,0.032898,0.044162
176,support,0.0100,5,0.067157,0.032898,0.044162
179,confidence,0.0100,5,0.067157,0.032898,0.044162
186,lift,0.0000,5,0.067157,0.032898,0.044162
180,confidence,0.0500,5,0.067157,0.032898,0.044162


,method,threshold,top_n,precision,recall,f1_score
173,support,0.0005,5,0.067157,0.032898,0.044162
187,lift,0.5000,5,0.067157,0.032898,0.044162
172,support,0.0001,5,0.067157,0.032898,0.044162
174,support,0.0010,5,0.067157,0.032898,0.044162
175,support,0.0050,5,0.067157,0.032898,0.044162
176,support,0.0100,5,0.067157,0.032898,0.044162
179,confidence,0.0100,5,0.067157,0.032898,0.044162
186,lift,0.0000,5,0.067157,0.032898,0.044162
180,confidence,0.0500,5,0.067157,0.032898,0.044162
199,conviction,0.5000,5,0.067157,0.032898,0.044162


,method,threshold,top_n,precision,recall,f1_score
9,confidence,0.1000,1,0.103226,0.014421,0.025306
4,support,0.0100,1,0.093366,0.017125,0.028941
8,confidence,0.0500,1,0.093366,0.017125,0.028941
7,confidence,0.0100,1,0.093366,0.017125,0.028941
0,support,0.0001,1,0.093366,0.017125,0.028941
3,support,0.0050,1,0.093366,0.017125,0.028941
2,support,0.0010,1,0.093366,0.017125,0.028941
1,support,0.0005,1,0.093366,0.017125,0.028941
95,confidence,0.1000,3,0.089130,0.018477,0.030608
52,confidence,0.1000,2,0.089130,0.018477,0.030608
